In [1]:
! wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
! tar xf ldcc-20140209.tar.gz
! rm ldcc-20140209.tar.gz

--2023-06-13 17:52:25--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
www.rondhuit.com (www.rondhuit.com) をDNSに問いあわせています... 59.106.19.174
www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 8855190 (8.4M) [application/x-gzip]
`ldcc-20140209.tar.gz' に保存中

ldcc-20140209.tar.g 100%[===================>]   8.44M  9.94MB/s    時間 0.8s    

2023-06-13 17:52:26 (9.94 MB/s) - `ldcc-20140209.tar.gz' へ保存完了 [8855190/8855190]



In [2]:
dirname2label = {
    'dokujo-tsushin': "独女通信",
    'it-life-hack': "ITライフハック",
    'kaden-channel': "家電チャンネル",
    'livedoor-homme': "livedoor HOMME",
    'movie-enter': "MOVIE ENTER",
    'peachy': "Peachy",
    'smax': "エスマックス",
    'sports-watch': "Sports Watch",
    'topic-news': "トピックニュース",
}

In [3]:
import os
import glob

data = []

for dirname, label in dirname2label.items():
    os.path.join('text', dirname)
    filepaths = glob.glob(os.path.join('text', dirname, '*.txt'))
    for filepath in filepaths:
        text = open(filepath, 'r', encoding='utf-8').read()
        # 内容のある最初の３文だけ取り出す
        lines = []
        for line in text.split('\n')[2:]:
            if len(line) > 0:
                lines.append(line)
                if len(lines) >= 3:
                    break
        text = '\n'.join(lines)
        text_with_label = f'{text}\nlabel: {label}\n'
        data.append({'text': text, 'text_with_label': text_with_label})


https://qiita.com/Mizuiro__sakura/items/058d2590d31e9f8aeeaa

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
MODEL_NAME="cyberagent/open-calm-medium"  # modelを選択してください
tokenizer=AutoTokenizer.from_pretrained(MODEL_NAME)
model=AutoModelForCausalLM.from_pretrained(MODEL_NAME)

/home/fujie/.conda/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
n_token = 512
from collections import defaultdict
import tqdm

def preprocess(examples, is_test=False):
    dataset = defaultdict(list)
    # original_sentenceは入力文章、generated_sentenceは出力してほしい文章を代入（入力文章部分も含む）
    for example in tqdm.tqdm(examples):
        original_sentence, generated_sentence = example["text"], example["text_with_label"]

        # for i, sentence in enumerate(generated_sentence):
        input = tokenizer(original_sentence, max_length=n_token, padding="max_length", truncation=True)["input_ids"]
        labels = tokenizer(generated_sentence, max_length=n_token, padding="max_length", truncation=True)["input_ids"]

        dataset["input_ids"].append(input)
        dataset["labels"].append(labels)

    return dataset

In [6]:
from torch.utils.data import Dataset, DataLoader

class Dataset(Dataset):
  def __init__(self, dataset, is_test=False):
    self.dataset = dataset
    self.is_test = is_test

  def __getitem__(self, idx):
    data = {'input_ids': torch.tensor(self.dataset["input_ids"][idx])}
    data['labels']=torch.tensor(self.dataset["labels"][idx])
    return data

  def __len__(self):
    return len(self.dataset["input_ids"])


In [7]:
dataset_train = Dataset(preprocess(data))

100%|██████████| 7376/7376 [00:07<00:00, 923.28it/s] 


In [8]:
from transformers import Trainer, TrainingArguments
training_config = TrainingArguments(
  output_dir = 'output_dir',  # 出力したいディレクトリを入力してください
  num_train_epochs = 4, 
  per_device_train_batch_size = 1,
  per_device_eval_batch_size = 1,
  warmup_steps = 100,
  weight_decay = 0.1,
  save_steps = 10000  # 用いるdatasetに応じてsave_stepsは変えてください
)

trainer = Trainer(
    model = model,                         
    args = training_config,
    train_dataset = dataset_train,
    
)

trainer.train()

/home/fujie/.conda/envs/py39/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,1.592200
1000,0.739900
1500,0.758900
2000,0.729000


KeyboardInterrupt: 